## 賃貸情報を整型して記述統計を表示する

## Googleドライブをマウントする

In [ ]:
from google.colab import drive
googledrive = '/content/drive'                                   # GoogleColaboratory の マウントポイント
drive.mount( googledrive )                                        # マウントポイントにGoogleDriveをマウントする
mydrive = googledrive + '/MyDrive'                           # 上でマウントしたGoogleDriveのMyDriveの絶対path
pylearn_dir = mydrive + "/Colab Notebooks/pylearn" # 空白がある場合は、ダブルクォーテーションで囲う
html_dir = mydrive + '/html'                                      # /content/drive/MyDrive/html HTML文書の置き場
suumo_dir = html_dir + '/suumo'                               # スーモ賃貸情報Googleシートの置き場

## Googleシートを読み込めるようにする

In [2]:
import gspread
from google.colab import auth
from google.colab import drive
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gss_key = '1FJB_uZsk89FVG9XddKwmOlIsmK_DHeCfwmLpdj0EUWM'
sheet = 'chintai'

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
workbook = gc.open_by_key(gss_key)
worksheet = workbook.worksheet(sheet)
worksheet_out = workbook.worksheet('temp')
worksheet_master = workbook.worksheet('master')
kinds_master = worksheet_master.col_values(1)      # 雑マスタの読み込み
layout_master = worksheet_master.col_values(2)     # 雑マスタの読み込み

## データを整型する

In [ ]:
import re

def get_kinds(field, kinds_master):
    for i, kinds in enumerate(kinds_master):
        if kinds == field:
            return i + 1
    return 0

print('タイプ', get_kinds('賃貸アパート', kinds_master))
print('タイプ', get_kinds('賃貸マンション', kinds_master))
print('タイプ', get_kinds('賃貸テラス・タウンハウス', kinds_master))
print('タイプ', get_kinds('賃貸一戸建て', kinds_master))
print('タイプ', get_kinds('賃貸その他', kinds_master))


In [ ]:
import re

def get_station(field):
    if re.search('.*駅 *歩\d{1,2}分.*', field) != None:
        matches = [ m.groups()[0] for m in re.finditer(r'.*駅 *歩(\d{1,2}).*', field, re.MULTILINE) ]
        return min(matches)
    else:
        return 0

print('最寄駅', get_station("東武伊勢崎線/竹ノ塚駅 バス25分 (バス停)保育所前 歩6分\n東武伊勢崎線/谷塚駅 歩32分\nつくばエクスプレス/八潮駅 歩33分"))
print('最寄駅', get_station("東武伊勢崎線/梅島駅 歩23分\n東武伊勢崎線/西新井駅 歩13分\n日暮里・舎人ライナー/江北駅 歩20分"))

In [ ]:
import re

def get_age(field):
     if re.match('築\d{1,2}年', field) != None:
        rep = field.replace('築', '')
        return list(map(int, re.match('(\d{1,2}).*', rep).groups()))[0]
     elif re.match('新築', field) != None:
        return 0

print('築年数', get_age("築5年"))
print('築年数', get_age("築35年"))
print('築年数', get_age("築15年"))

In [ ]:
import re

def get_numoffloors(field):
    if re.match('\d{1,2}階建', field) != None:
        return list(map(int, re.match('(\d{1,2}).*', field).groups()))[0]
    else:
        return 0

print('階数', get_numoffloors('1階建'))
print('階数', get_numoffloors('8階建'))
print('階数', get_numoffloors('18階建'))

In [ ]:
import re

def get_floor(field):
    if re.match('\d{1,2}階', field) != None:
        return list(map(int, re.match('(\d{1,2}).*', field).groups()))[0]
    else:
        return 0

print('階', get_floor('17階'))
print('階', get_floor('3階'))
print('階', get_floor('1階'))

In [ ]:
import re

def get_rent(field):
    if re.match('\d{1,3}\.\d{1,2}万円', field) != None:
        return int(list(map(float, re.match('(\d{1,3}\.\d{1,2}).*', field).groups()))[0] * 10000)
    if re.match('\d{1,3}万円', field) != None:
        return list(map(int, re.match('(\d{1,3}).*', field).groups()))[0] * 10000
    else:
        return 0

print('賃料', get_rent('12万円'))
print('賃料', get_rent('8.2万円'))
print('賃料', get_rent('17.2万円'))
print('賃料', get_rent('7万円'))
print('賃料', get_rent('107万円'))

In [ ]:
import re

def get_fee(field):
    if re.match('\d{4,7}円', field) != None:
        return list(map(int, re.match('(\d{4,7}).*', field).groups()))[0]
    else:
        return 0

print('管理・共益費', get_fee('12000円'))
print('管理・共益費', get_fee('3000円'))

In [ ]:
import re

def get_reward(field):
    if re.match('\d{1,3}\.\d{1,2}万円', field) != None:
        return int(list(map(float, re.match('(\d{1,3}\.\d{1,2}).*', field).groups()))[0] * 10000)
    if re.match('\d{1,3}万円', field) != None:
        return list(map(int, re.match('(\d{1,3}).*', field).groups()))[0] * 10000
    else:
        return 0

print('敷金・礼金', get_reward('12万円'))
print('敷金・礼金', get_reward('1.2万円'))
print('敷金・礼金', get_reward('-'))

In [ ]:
import re

def get_layout(field, layout_master):
    for i, layout in enumerate(layout_master):
        if layout == field:
            return i + 1
    return 0

print('間取り', get_layout('ワンルーム', layout_master))
print('間取り', get_layout('1K', layout_master))
print('間取り', get_layout('1SK', layout_master))
print('間取り', get_layout('1DK', layout_master))
print('間取り', get_layout('1LDK', layout_master))
print('間取り', get_layout('1SDK', layout_master))
print('間取り', get_layout('1SLDK', layout_master))
print('間取り', get_layout('2K', layout_master))
print('間取り', get_layout('2DK', layout_master))
print('間取り', get_layout('2SDK', layout_master))
print('間取り', get_layout('2LDK', layout_master))
print('間取り', get_layout('2SLDK', layout_master))
print('間取り', get_layout('3K', layout_master))
print('間取り', get_layout('3SK', layout_master))
print('間取り', get_layout('3DK', layout_master))
print('間取り', get_layout('3LDK', layout_master))
print('間取り', get_layout('3SLDK', layout_master))
print('間取り', get_layout('4K', layout_master))
print('間取り', get_layout('4DK', layout_master))
print('間取り', get_layout('4SDK', layout_master))
print('間取り', get_layout('4LDK', layout_master))
print('間取り', get_layout('4SLDK', layout_master))
print('間取り', get_layout('5DK', layout_master))
print('間取り', get_layout('5LDK', layout_master))
print('間取り', get_layout('5SLDK', layout_master))
print('間取り', get_layout('6K', layout_master))
print('間取り', get_layout('6LDK', layout_master))
print('間取り', get_layout('7SDK', layout_master))
print('間取り', get_layout('8LDK', layout_master))


In [13]:
import re

def get_square(field):
    if re.match('\d{1,3}\.\d{1,2}m2', field) != None:
        return list(map(float, re.match('(\d{1,3}\.\d{1,2}).*', field).groups()))[0]
    if re.match('\d{1,3}m2', field) != None:
        return list(map(int, re.match('(\d{1,3}).*', field).groups()))[0]
    else:
        return 0

print('平米', get_square('26m2'))
print('平米', get_square('15.34m2'))
print('平米', get_square('126m2'))
print('平米', get_square('5.34m2'))
print('平米', get_square('105.34m2'))

平米 26
平米 15.34
平米 126
平米 5.34
平米 105.34


In [ ]:
values = worksheet.get_all_values()

import pandas as pd
from gspread_dataframe import set_with_dataframe

pd.options.display.float_format = '{:}'.format

df = pd.DataFrame.from_records(data=values[1:])
df = df.astype({
    0: str, 1: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str, 8: str, 9: str,
    10: str, 11: str, 12: str, 13: str
})
# A=0, B=1, C=2, D=3, E=4, F=5, G=6, H=7, I=8, J=9, K=10, L=11, M=12,  N=13

for i, elm in df.iterrows():
    # if i < 0:
    #     continue
    kinds = get_kinds(elm[1], kinds_master)
    title = elm[2]
    address = elm[3]
    station = get_station(elm[4])
    age = get_age(elm[5])
    numoffloors = get_numoffloors(elm[6])
    floor = get_floor(elm[7])
    rent = get_rent(elm[8])
    fee = get_fee(elm[9])
    reward = get_reward(elm[10])
    layout = get_layout(elm[11], layout_master)
    square = get_square(elm[12])

    sheet_range = 'B' + str( i + 2 ) + ':N' + str( i + 2 )
    cell_list = worksheet_out.range( sheet_range )
    cell_list[0].value = kinds
    cell_list[1].value = title
    cell_list[2].value = address
    cell_list[3].value = station
    cell_list[4].value = age
    cell_list[5].value = numoffloors
    cell_list[6].value = floor
    cell_list[7].value = rent
    cell_list[8].value = fee
    cell_list[9].value = reward
    cell_list[10].value = layout
    cell_list[11].value = square
    worksheet_out.update_cells(cell_list)


## 統計情報を出力

- 間取りごとの平均単価

In [ ]:
import gspread
from google.colab import auth
from google.colab import drive
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gss_key = '1FJB_uZsk89FVG9XddKwmOlIsmK_DHeCfwmLpdj0EUWM'
sheet = 'temp'

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
workbook = gc.open_by_key(gss_key)
worksheet = workbook.worksheet(sheet)
worksheet_master = workbook.worksheet('master')
kinds_master = worksheet_master.col_values(1)      # 雑マスタの読み込み
layout_master = worksheet_master.col_values(2)     # 雑マスタの読み込み

import numpy as np
import pandas as pd
from gspread_dataframe import set_with_dataframe

values = worksheet.get_all_values()
pd.options.display.float_format = '{:}'.format

df = pd.DataFrame.from_records(data=values[1:])
df = df.astype({
    0: str, 1: int, 2: str, 3: str, 4: int, 5: int, 6: int, 7: int, 8: int, 9: int,
    10: int, 11: int, 12: float
})
# A=0, B=1, C=2, D=3, E=4, F=5, G=6, H=7, I=8, J=9, K=10, L=11, M=12

df['monthly'] = df[8] + df[9]
print(df.groupby(11)['monthly'].describe())

- 24ヶ月間の1㎡（平米）あたりの1ヶ月ごとの平均単価

In [ ]:
import gspread
from google.colab import auth
from google.colab import drive
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gss_key = '1FJB_uZsk89FVG9XddKwmOlIsmK_DHeCfwmLpdj0EUWM'
sheet = 'temp'

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
workbook = gc.open_by_key(gss_key)
worksheet = workbook.worksheet(sheet)
worksheet_master = workbook.worksheet('master')
kinds_master = worksheet_master.col_values(1)      # 雑マスタの読み込み
layout_master = worksheet_master.col_values(2)     # 雑マスタの読み込み

import numpy as np
import pandas as pd
from gspread_dataframe import set_with_dataframe

values = worksheet.get_all_values()
pd.options.display.float_format = '{:}'.format

df = pd.DataFrame.from_records(data=values[1:])
df = df.astype({
    0: str, 1: int, 2: str, 3: str, 4: int, 5: int, 6: int, 7: int, 8: int, 9: int,
    10: int, 11: int, 12: float
})
# A=0, B=1, C=2, D=3, E=4, F=5, G=6, H=7, I=8, J=9, K=10, L=11, M=12

df['rent'] = df[8] + df[9] + (df[10] / 24)
print(df['rent'].describe())
print('\n')
df['@m2'] = ( df[8] + df[9]  + ( df[10] / 24 ) ) / df[12]
print(df['@m2'].describe())
print('\n')
df['m2'] = df[12]
print(df['m2'].describe())

## 散布図を表示する

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import gspread
from google.colab import auth
from google.colab import drive
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gss_key = '1FJB_uZsk89FVG9XddKwmOlIsmK_DHeCfwmLpdj0EUWM'
sheet = 'temp'

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
workbook = gc.open_by_key(gss_key)
worksheet = workbook.worksheet(sheet)
worksheet_master = workbook.worksheet('master')
kinds_master = worksheet_master.col_values(1)      # 雑マスタの読み込み
layout_master = worksheet_master.col_values(2)     # 雑マスタの読み込み

import numpy as np
import pandas as pd
from gspread_dataframe import set_with_dataframe

values = worksheet.get_all_values()
pd.options.display.float_format = '{:}'.format

df = pd.DataFrame.from_records(data=values[1:])
df = df.astype({
    0: str, 1: int, 2: str, 3: str, 4: int, 5: int, 6: int, 7: int, 8: int, 9: int,
    10: int, 11: int, 12: float
})
# A=0, B=1, C=2, D=3, E=4, F=5, G=6, H=7, I=8, J=9, K=10, L=11, M=12
df['@mon'] = df[8] + df[9]  + ( df[10] / 24 )

df.plot.scatter(x=1, y='@mon', alpha=0.5) # 建物のタイプ
df.plot.scatter(x=12, y='@mon', alpha=0.5) # 平米
df.plot.scatter(x=11, y='@mon', alpha=0.5) # 間取り
df.plot.scatter(x=4, y='@mon', alpha=0.5) # 最寄りからの徒歩所要時間
df.plot.scatter(x=5, y='@mon', alpha=0.5) # 築年数
df.plot.scatter(x=6, y='@mon', alpha=0.5) # 建物自体の階数
